<a href="https://colab.research.google.com/github/mca2309/OCR-using-OpenCV-and-Tensorflow-with-GUI/blob/R2/OCR_on_3D_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import cv2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import exists
import h5py

In [2]:
#Downloading and extracting the CIFAKE dataset from kaggle
from google.colab import drive
drive.mount('/content/drive')

!pip install kaggle

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

!kaggle datasets download -d daavoo/3d-mnist

import zipfile

file_path = '/content/3d-mnist.zip'

!mkdir 3d-mnist

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/3d-mnist')

Mounted at /content/drive
 90% 137M/153M [00:01<00:00, 130MB/s]
100% 153M/153M [00:01<00:00, 131MB/s]


In [20]:
# Load the dataset
f = h5py.File("/content/3d-mnist/full_dataset_vectors.h5", "r")
trainX = f["X_train"][:]
trainY = f["y_train"][:]
testX = f["X_test"][:]
testY = f["y_test"][:]

In [21]:
# Reshape the flattened images into 2D arrays
trainX = trainX.reshape((trainX.shape[0], 64, 64, 1))
testX = testX.reshape((testX.shape[0], 64, 64, 1))

In [22]:
# Convert pixel values to float and scale them to the range of 0-1
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

In [23]:
# One-hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

print("Train dataset shape:", trainX.shape)
print("Test dataset shape:", testX.shape)


Train dataset shape: (10000, 64, 64, 1)
Test dataset shape: (2000, 64, 64, 1)


In [24]:
# scale pixels
train_norm = trainX.astype('float32')
test_norm = testX.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0
# return normalized images
trainX = train_norm
testX = test_norm

In [28]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

In [29]:
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit our model
history = model.fit(trainX, trainY, epochs=10, validation_data=(testX, testY))
# save our model
model.save('mnist_3D_cnn.h5')

Epoch 1/10
313/313 [==============================] - 41s 128ms/step - loss: 2.3015 - accuracy: 0.1086 - val_loss: 2.2990 - val_accuracy: 0.1260
Epoch 2/10
 86/313 [=======>......................] - ETA: 27s - loss: 2.2989 - accuracy: 0.1057